Westbrook:defensive rebound to an unassisted basket within five seconds and within three feet of the rim  
Unseld: defensive rebound to an assist within three seconds and within three feet of the rim  
Chase-down: block within five seconds of a possession change and within five feet of the rim  

In [1]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/pbp_related/"
source = "pbpstats"

In [ ]:
league = "NBA"
year = 2022
season_type = "Regular Season"
if league == "NBA":
    season = str(year) + "-" + str(year + 1)[-2:]
else:
    season = str(year)
player_dict = get_players_pbp(league=league)  # Creating Player Dictionary
team_dict = nba_teams.get_teams()  # Creating Team Dictionary
games_id = pbp_season(league=league, season_yr=str(year), season_type=season_type)
games_list = pbp_games(games_id)

In [ ]:
# years = [2017,2018,2019,2020,2021,2022,2023]
# games_list = []
# for year in years:
#     games_id = pbp_season(league=league,season_yr=str(year),season_type=season_type)
#     games_list1 = pbp_games(games_id)
#     games_list += games_list1
#     fp.close()

# Chasedown Blocks

In [ ]:
possessions = [game.possessions.items for game in games_list]
possession_events = list(chain(*[possession.events for possession in list(chain(*possessions))]))
t = 5
dist = 5
Block_pID, FGA_pID = [], []
for possession_event in possession_events:
    if (
        (
            isinstance(possession_event, FieldGoal)
            and not possession_event.is_made
        )
        and possession_event.is_blocked
        and possession_event.seconds_since_previous_event <= t
        and possession_event.distance <= dist
    ):
        if (
            isinstance(possession_event.previous_event, Rebound)
            and possession_event.previous_event.oreb
        ):
            pass
        else:
            Block_pID.append(possession_event.player3_id)
            FGA_pID.append(possession_event.player1_id)

In [ ]:
data_cb = rank_data_pbp(Block_pID, player_dict, team_dict, sort="Player", var="BLK")

In [ ]:
title = "Chasedown Blocks " + season
footer = "Block <5 feet of rim and within 5s of poss change"
fig = plot_table_rank(data_cb, "BLK", title=title, footer=footer, source="pbpstats")
fig.write_image(fig_DIR + f"{league}_chasedown_{year}.png", scale=3)